In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load both datasets
iig_df = pd.read_csv(r'C:\Users\GaganReddy\projects\climatewise\Projects\Actual\Analysis\projectID Based\iig_data\iig_cleaned_dataset.csv')
etim_df = pd.read_csv(r'C:\Users\GaganReddy\projects\climatewise\Projects\Actual\Analysis\projectID Based\etim_data\combined_cleaned_dataset.csv')

# Basic info about both datasets
print("IIG Dataset Overview:")
print(f"Shape: {iig_df.shape}")
print(f"Columns: {list(iig_df.columns)}")

print("\nETIM Dataset Overview:")
print(f"Shape: {etim_df.shape}")
print(f"Columns: {list(etim_df.columns)}")

# Check column overlap
iig_cols = set(iig_df.columns)
etim_cols = set(etim_df.columns)
common_cols = iig_cols.intersection(etim_cols)

print(f"\nCommon columns between datasets: {len(common_cols)}")
print("Common columns:", common_cols)

# Compare project size distributions
if 'Investment (USD Million)' in etim_df.columns and 'Total Project Cost (USD mn)' in iig_df.columns:
    # Convert to numeric, handling errors
    etim_df['Investment (USD Million)'] = pd.to_numeric(etim_df['Investment (USD Million)'], errors='coerce')
    iig_df['Total Project Cost (USD mn)'] = pd.to_numeric(iig_df['Total Project Cost (USD mn)'], errors='coerce')
    
    # Create figure
    fig = make_subplots(rows=1, cols=2, subplot_titles=("ETIM Dataset", "IIG Dataset"))
    
    # Add histograms
    fig.add_trace(
        go.Histogram(x=etim_df['Investment (USD Million)'], 
                      name="ETIM", 
                      nbinsx=20,
                      marker_color='blue',
                      opacity=0.7),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Histogram(x=iig_df['Total Project Cost (USD mn)'], 
                      name="IIG", 
                      nbinsx=20,
                      marker_color='green',
                      opacity=0.7),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text="Project Cost Distribution Comparison",
        template="plotly_white",
        height=500,
        width=1000
    )
    
    fig.show()

# Compare sectors/categories
if 'source' in etim_df.columns and 'Sector' in iig_df.columns:
    # Get sector distributions
    etim_sectors = etim_df['source'].value_counts().reset_index()
    etim_sectors.columns = ['Sector', 'Count']
    etim_sectors['Dataset'] = 'ETIM'
    
    iig_sectors = iig_df['Sector'].value_counts().reset_index()
    iig_sectors.columns = ['Sector', 'Count']
    iig_sectors['Dataset'] = 'IIG'
    
    # Combine data
    sector_comparison = pd.concat([etim_sectors, iig_sectors], ignore_index=True)
    
    # Plot
    fig = px.bar(
        sector_comparison,
        x='Sector',
        y='Count',
        color='Dataset',
        barmode='group',
        title='Sector Distribution Comparison',
        height=500
    )
    
    fig.update_layout(template="plotly_white")
    fig.show()

# Compare geographic distribution
if 'State' in etim_df.columns and 'State' in iig_df.columns:
    # Clean state names (this might need customization based on the actual data)
    etim_df['State'] = etim_df['State'].fillna('Unknown')
    iig_df['State'] = iig_df['State'].fillna('Unknown')
    
    # Get state distributions
    etim_states = etim_df['State'].value_counts().head(10).reset_index()
    etim_states.columns = ['State', 'Count']
    etim_states['Dataset'] = 'ETIM'
    
    iig_states = iig_df['State'].value_counts().head(10).reset_index()
    iig_states.columns = ['State', 'Count']
    iig_states['Dataset'] = 'IIG'
    
    # Combine data for top states
    all_top_states = set(etim_states['State']).union(set(iig_states['State']))
    
    # Plot
    fig = px.bar(
        pd.concat([etim_states, iig_states], ignore_index=True),
        x='State',
        y='Count',
        color='Dataset',
        barmode='group',
        title='Top States Comparison',
        height=600
    )
    
    fig.update_layout(
        template="plotly_white",
        xaxis={'categoryorder':'total descending'}
    )
    fig.show()

# Find large projects (>$500M) in both datasets
large_etim_projects = etim_df[etim_df['Investment (USD Million)'] > 500].copy()
large_iig_projects = iig_df[iig_df['Total Project Cost (USD mn)'] > 500].copy()

print(f"\nLarge projects (>$500M) in ETIM dataset: {len(large_etim_projects)}")
print(f"Large projects (>$500M) in IIG dataset: {len(large_iig_projects)}")

# Compare project status distributions
if 'Project Status' in etim_df.columns and 'Project Status' in iig_df.columns:
    etim_status = etim_df['Project Status'].value_counts().reset_index()
    etim_status.columns = ['Status', 'Count']
    etim_status['Dataset'] = 'ETIM'
    
    iig_status = iig_df['Project Status'].value_counts().reset_index()
    iig_status.columns = ['Status', 'Count']
    iig_status['Dataset'] = 'IIG'
    
    # Combine
    status_comparison = pd.concat([etim_status, iig_status], ignore_index=True)
    
    # Plot
    fig = px.bar(
        status_comparison,
        x='Status',
        y='Count',
        color='Dataset',
        barmode='group',
        title='Project Status Comparison',
        height=500
    )
    
    fig.update_layout(template="plotly_white")
    fig.show()

# Identify potential overlaps (projects that might be in both datasets)
# This is an approximation - you'd need more sophisticated matching for a real analysis
if 'Developersorted ascending' in etim_df.columns and 'Developer' in iig_df.columns:
    etim_developers = set(etim_df['Developersorted ascending'].dropna())
    iig_developers = set(iig_df['Developer'].dropna())
    common_developers = etim_developers.intersection(iig_developers)
    
    print(f"\nPotential overlap - developers in both datasets: {len(common_developers)}")
    if len(common_developers) > 0:
        print("Sample common developers:", list(common_developers)[:5])

IIG Dataset Overview:
Shape: (6725, 10)
Columns: ['Project ID', 'Title', 'Sector', 'Subsector', 'State', 'Start Date', 'Completion Date', 'Total Project Cost (USD mn)', 'Project Status', 'Source URL']

ETIM Dataset Overview:
Shape: (1557, 12)
Columns: ['Contact Developer', 'Verfified by developer', 'Investment (USD Million)', 'Developersorted ascending', 'Developersorted ascendingDeveloper Category', 'Planned COD', 'Unnamed: 1', 'Developer Category', 'Project Status', 'State', 'Project Name', 'source']

Common columns between datasets: 2
Common columns: {'State', 'Project Status'}



Large projects (>$500M) in ETIM dataset: 75
Large projects (>$500M) in IIG dataset: 134


In [7]:
# Add this cell below the existing comparison code

import pandas as pd
import json

# Load both datasets
iig_df = pd.read_csv(r'C:\Users\GaganReddy\projects\climatewise\Projects\Actual\Analysis\projectID Based\iig_data\iig_cleaned_dataset.csv')
etim_df = pd.read_csv(r'C:\Users\GaganReddy\projects\climatewise\Projects\Actual\Analysis\projectID Based\etim_data\combined_cleaned_dataset.csv')

# Function to organize developers by state and project type
def organize_developers(df, developer_col, location_col, type_col):
    # Create a dictionary to store the organized data
    organized_data = {}
    
    # Handle missing values for better grouping
    df[location_col] = df[location_col].fillna('Unknown')
    df[type_col] = df[type_col].fillna('Unknown')
    df[developer_col] = df[developer_col].fillna('Unnamed Developer')
    
    # Group by location and project type
    for location in sorted(df[location_col].unique()):
        if location == '--' or pd.isna(location):
            location = 'Unknown'
            
        organized_data[location] = {}
        
        # Filter for this location
        location_df = df[df[location_col] == location]
        
        for proj_type in sorted(location_df[type_col].unique()):
            # Filter for this project type within this location
            type_df = location_df[location_df[type_col] == proj_type]
            
            # Get unique developers for this location and project type
            developers = sorted(type_df[developer_col].unique())
            
            # Add additional info like count and average project size
            projects_count = len(type_df)
            
            # Try to get average project size if the column exists
            try:
                if 'Investment (USD Million)' in type_df.columns:
                    avg_size = pd.to_numeric(type_df['Investment (USD Million)'], errors='coerce').mean()
                elif 'Total Project Cost (USD mn)' in type_df.columns:
                    avg_size = pd.to_numeric(type_df['Total Project Cost (USD mn)'], errors='coerce').mean()
                else:
                    avg_size = 'N/A'
            except:
                avg_size = 'N/A'
                
            # Store in the dictionary with additional info
            organized_data[location][proj_type] = {
                'developers': developers,
                'count': projects_count,
                'avg_project_size': avg_size if not pd.isna(avg_size) else 'N/A'
            }
    
    return organized_data

# Organize ETIM developers
etim_developers = organize_developers(
    etim_df, 
    'Developersorted ascending', 
    'State', 
    'source'
)

# Organize IIG developers
iig_developers = organize_developers(
    iig_df,
    'Developer',  # Assuming this is the column name
    'State', 
    'Sector'
)

# Function to create a formatted output
def format_output(organized_data, dataset_name):
    output = f"# {dataset_name} Developers by Location and Project Type\n\n"
    
    for location, types in organized_data.items():
        output += f"## {location}\n\n"
        
        for proj_type, data in types.items():
            developer_count = len(data['developers'])
            output += f"### {proj_type} ({data['count']} projects, Avg size: "
            
            if isinstance(data['avg_project_size'], float):
                output += f"${data['avg_project_size']:.2f}M)\n\n"
            else:
                output += f"{data['avg_project_size']})\n\n"
                
            for i, developer in enumerate(data['developers']):
                output += f"{i+1}. {developer}\n"
            
            output += "\n"
    
    return output

# Generate and print the formatted outputs
etim_output = format_output(etim_developers, "ETIM Dataset")
print(etim_output)

iig_output = format_output(iig_developers, "IIG Dataset")
print(iig_output)

# Save to JSON files for potential use in the dashboard
with open('etim_developers_by_location_type.json', 'w') as f:
    json.dump(etim_developers, f, indent=2)

with open('iig_developers_by_location_type.json', 'w') as f:
    json.dump(iig_developers, f, indent=2)

# Also save as CSV files with hierarchical structure
def save_as_csv(organized_data, filename):
    rows = []
    for location, types in organized_data.items():
        for proj_type, data in types.items():
            for developer in data['developers']:
                rows.append({
                    'Location': location,
                    'Project Type': proj_type,
                    'Developer': developer,
                    'Project Count': data['count'],
                    'Avg Project Size (USD M)': data['avg_project_size'] if isinstance(data['avg_project_size'], float) else 'N/A'
                })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)

save_as_csv(etim_developers, 'etim_developers_by_location_type.csv')
save_as_csv(iig_developers, 'iig_developers_by_location_type.csv')

KeyError: 'Developer'

In [8]:
# First, let's check the actual column names in both datasets
print("ETIM Columns:", etim_df.columns.tolist())
print("IIG Columns:", iig_df.columns.tolist())

# Based on the column inspection, let's organize developers with the correct column names
import pandas as pd
import json

# Function to organize developers by state and project type
def organize_developers(df, developer_col, location_col, type_col):
    # Create a dictionary to store the organized data
    organized_data = {}
    
    # Make sure the column names exist in the dataframe
    if developer_col not in df.columns:
        print(f"Error: '{developer_col}' column not found in dataframe")
        return organized_data
        
    if location_col not in df.columns:
        print(f"Error: '{location_col}' column not found in dataframe")
        return organized_data
        
    if type_col not in df.columns:
        print(f"Error: '{type_col}' column not found in dataframe")
        return organized_data
    
    # Handle missing values for better grouping
    df[location_col] = df[location_col].fillna('Unknown')
    df[type_col] = df[type_col].fillna('Unknown')
    df[developer_col] = df[developer_col].fillna('Unnamed Developer')
    
    # Group by location and project type
    for location in sorted(df[location_col].unique()):
        if location == '--' or pd.isna(location):
            location = 'Unknown'
            
        organized_data[location] = {}
        
        # Filter for this location
        location_df = df[df[location_col] == location]
        
        for proj_type in sorted(location_df[type_col].unique()):
            # Filter for this project type within this location
            type_df = location_df[location_df[type_col] == proj_type]
            
            # Get unique developers for this location and project type
            developers = sorted(type_df[developer_col].unique())
            
            # Add additional info like count and average project size
            projects_count = len(type_df)
            
            # Try to get average project size if the column exists
            try:
                if 'Investment (USD Million)' in type_df.columns:
                    avg_size = pd.to_numeric(type_df['Investment (USD Million)'], errors='coerce').mean()
                elif 'Total Project Cost (USD mn)' in type_df.columns:
                    avg_size = pd.to_numeric(type_df['Total Project Cost (USD mn)'], errors='coerce').mean()
                else:
                    avg_size = 'N/A'
            except:
                avg_size = 'N/A'
                
            # Store in the dictionary with additional info
            organized_data[location][proj_type] = {
                'developers': developers,
                'count': projects_count,
                'avg_project_size': avg_size if not pd.isna(avg_size) else 'N/A'
            }
    
    return organized_data

# Organize ETIM developers
etim_developers = organize_developers(
    etim_df, 
    'Developersorted ascending', 
    'State', 
    'source'
)

# For IIG, we need to determine the correct column name for developers
# Let's use a safer approach by checking what's actually in the dataframe
developer_col_iig = None
possible_developer_cols = ['Developer', 'Developersorted ascending', 'Project Developer', 'Project Sponsor']

for col in possible_developer_cols:
    if col in iig_df.columns:
        developer_col_iig = col
        print(f"Using '{col}' as developer column for IIG dataset")
        break

if developer_col_iig is None:
    print("Could not find a suitable developer column in IIG dataset")
    print("Available columns:", iig_df.columns.tolist())
else:
    # Organize IIG developers
    iig_developers = organize_developers(
        iig_df,
        developer_col_iig,
        'State' if 'State' in iig_df.columns else 'Location', 
        'Sector' if 'Sector' in iig_df.columns else 'Project Type'
    )

    # Function to create a formatted output
    def format_output(organized_data, dataset_name):
        output = f"# {dataset_name} Developers by Location and Project Type\n\n"
        
        for location, types in organized_data.items():
            output += f"## {location}\n\n"
            
            for proj_type, data in types.items():
                developer_count = len(data['developers'])
                output += f"### {proj_type} ({data['count']} projects, Avg size: "
                
                if isinstance(data['avg_project_size'], float):
                    output += f"${data['avg_project_size']:.2f}M)\n\n"
                else:
                    output += f"{data['avg_project_size']})\n\n"
                    
                for i, developer in enumerate(data['developers']):
                    output += f"{i+1}. {developer}\n"
                
                output += "\n"
        
        return output

    # Generate and print the formatted outputs
    etim_output = format_output(etim_developers, "ETIM Dataset")
    print(etim_output)

    iig_output = format_output(iig_developers, "IIG Dataset")
    print(iig_output)

    # Save to JSON files for potential use in the dashboard
    with open('etim_developers_by_location_type.json', 'w') as f:
        json.dump(etim_developers, f, indent=2)

    with open('iig_developers_by_location_type.json', 'w') as f:
        json.dump(iig_developers, f, indent=2)

    # Also save as CSV files with hierarchical structure
    def save_as_csv(organized_data, filename):
        rows = []
        for location, types in organized_data.items():
            for proj_type, data in types.items():
                for developer in data['developers']:
                    rows.append({
                        'Location': location,
                        'Project Type': proj_type,
                        'Developer': developer,
                        'Project Count': data['count'],
                        'Avg Project Size (USD M)': data['avg_project_size'] if isinstance(data['avg_project_size'], float) else 'N/A'
                    })
        
        df = pd.DataFrame(rows)
        df.to_csv(filename, index=False)

    save_as_csv(etim_developers, 'etim_developers_by_location_type.csv')
    save_as_csv(iig_developers, 'iig_developers_by_location_type.csv')

ETIM Columns: ['Contact Developer', 'Verfified by developer', 'Investment (USD Million)', 'Developersorted ascending', 'Developersorted ascendingDeveloper Category', 'Planned COD', 'Unnamed: 1', 'Developer Category', 'Project Status', 'State', 'Project Name', 'source']
IIG Columns: ['Project ID', 'Title', 'Sector', 'Subsector', 'State', 'Start Date', 'Completion Date', 'Total Project Cost (USD mn)', 'Project Status', 'Source URL']
Could not find a suitable developer column in IIG dataset
Available columns: ['Project ID', 'Title', 'Sector', 'Subsector', 'State', 'Start Date', 'Completion Date', 'Total Project Cost (USD mn)', 'Project Status', 'Source URL']
